<a href="https://colab.research.google.com/github/12jeong/Kaggle-beginner/blob/master/Santander-Product-Recommendation-Competition/D03_model_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from tensorflow.python.client import device_lib
import platform
from google.colab import drive
drive.mount('/content/drive/')

In [0]:
import os
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# For Jupyter Notebook
%matplotlib inline
import seaborn as sns
import xgboost as xgb
import pickle
os.getcwd()
os.chdir('/content/drive/My Drive/Colab Notebooks/Kaggle-beginner')

In [0]:
filename = "feature_list.pickle"
with open(filename, 'rb') as f:
    features = pickle.load(f)
    prods = pickle.load(f) 

In [0]:
df_trn = pd.read_csv('df_trn.csv') # 4GB

### 3. baseline 모형 학습 
- train_ver2.csv : 2015-01-28~2016-05-28  
- test_ver2.csv : 2016-06-28
- baseline 용 간소화된 모델을 위해 2016-01-28~2016-04-28 을 학습 데이터, 2016-05-28 을 검증 데이터로 사용

In [0]:
# 데이터 분리
use_dates = ['2016-01-28','2016-02-28','2016-03-28','2016-04-28','2016-05-28']
trn = df_trn[df_trn['fecha_dato'].isin(use_dates)]
tst = df_trn[df_trn['fecha_dato']=='2016-06-28']
del df_trn

In [92]:
# 참고
lst = ['dog','cat','bird','monkey']
for x, y in enumerate(lst):
    print(x)
    print(y)

0
dog
1
cat
2
bird
3
monkey


In [0]:
# 신규 상품 구매 정보가 있는 데이터만 추출
X = []
Y = []
for i,prod in enumerate(prods):
    prev = prod+'_prev'
    prX = trn[(trn[prod]==1) & (trn[prev]==0)]   # 이 방법 매우 좋은듯 
    prY = np.zeros(prX.shape[0], dtype=np.int8) + i # 몇 번째 상품이 신규구매인지 표시 
    X.append(prX)
    Y.append(prY)
XY = pd.concat(X) # 가로 쌓기
Y = np.hstack(Y)  # 배열 옆으로 쌓기 ...? 
XY['y'] = Y

In [91]:
Y # 24개의 class를 예측하는 문제

array([ 0,  1,  2, ..., 23, 23, 23], dtype=int8)

In [0]:
# 데이터 분리
vld_date = '2016-05-28'
XY_trn = XY[XY['fecha_dato']!=vld_date]
XY_vld = XY[XY['fecha_dato']==vld_date]

XGBoost
- Gradient Tree Boosting with Regularization (variant of original GBM)
 = XGBoost (eXtreme Gradient Boosting)
 $$ \hat{y_i} = \sum_{k=1}^K f_k (x_i) , ~~f_k \in \mathcal{F} $$
 이때 K는 트리의 수이고, 목적함수는 다음과 같음 (자세한 설명은 생략)
 $$ \mbox{min} \sum_{i=1}^n l (y_i, \hat{y_i}) + \sum_{k=1}^K \Omega (f_k) $$

- 모델 파라미터
    - est : learning late
    - gamma : Information Gain의 -r
    - max_depth : 트리의 최대 깊이
    - colsample_bytree : 변수 샘플링 비율
    - colsample_bylevel : 트리 레벨 별 변수 샘플링 비율

- (참고) 파이썬 사이킷런에서 제공하지 않음
- 시간 투자 대비 효율을 생각했을 때 파라미터 튜닝 보다 피처 엔지니어링에 더 신경쓰는 것을 권장하는 의견이 있음


In [0]:
# specify parameters via map
param = {
    'booster': 'gbtree',
    'max_depth': 8,
    'nthread': 4,
    'num_class': len(prods),
    'objective': 'multi:softprob',
    'silent': 1,
    'eval_metric': 'mlogloss',
    'eta': 0.1,
    'min_child_weight': 10,
    'colsample_bytree': 0.8,
    'colsample_bylevel': 0.9,
    'seed': 2018,
}

# read in data
X_trn = XY_trn.as_matrix(columns=features)
Y_trn = XY_trn.as_matrix(columns=['y'])
dtrn = xgb.DMatrix(X_trn, label=Y_trn, feature_names=features)

X_vld = XY_vld.as_matrix(columns=features)
Y_vld = XY_vld.as_matrix(columns=['y'])
dvld = xgb.DMatrix(X_vld, label=Y_vld, feature_names=features)

> **class xgboost.DMatrix**(data, label=None, weight=None, base_margin=None, missing=None, silent=False, feature_names=None, feature_types=None, nthread=None)
무슨 역할인지 아직 못찾음. 

> **xgboost.train**(params, dtrain, num_boost_round=10, evals=(), obj=None, feval=None, maximize=False, early_stopping_rounds=None, evals_result=None, verbose_eval=True, xgb_model=None, callbacks=None)  
params (dict) – Booster params.  
dtrain (DMatrix) – Data to be trained.   
num_boost_round (int) – Number of boosting iterations.  
evals (list of pairs (DMatrix, string)) – List of validation sets for which metrics will evaluated during training. Validation metrics will help us track the performance of the model. Example: with a watchlist containing [(dtest,'eval'), (dtrain,'train')]

In [0]:
watch_list = [(dtrn,'train'),(dvld,'eval')] # evlas의 역할? 왜 validation matrix를 넣지?
model = xgb.train(param, dtrn, num_boost_round=1000, evals=watch_list, early_stopping_rounds= 20) 

In [0]:
# 모델 저장
# filename = 'xgb.baseline.pkl'
# pickle.dump(model, open(filename,'wb'))
best_ntree_limit = model.best_ntree_limit # ?

In [93]:
best_ntree_limit # iteration이 멈춘 시점?

293

In [0]:
## Cross validation - MAP@7 계산  (baseline model)
vld = trn[trn['fecha_dato']==vld_date]
ncodpers_vld = vld.as_matrix(columns=['ncodpers'])
# 신규 구매 정갑값 추출
for prod in prods:
    prev = prod + '_prev'
    padd = prod + '_add'
    vld[padd] = vld[prod] - vld[prev] # 신규구매면 1
add_vld = vld.as_matrix(columns=[prod+'_add' for prod in prods])
add_vld_list = [list() for i in range(len(ncodpers_vld))]

# add_vld_list: 고객 별 신규 구매 정답값
count_vld = 0 
for ncodpers in range(len(ncodpers_vld)):
    for prod in range(len(prods)):
        if add_vld[ncodpers, prod] >0:
            add_vld_list[ncodpers].append(prod)
            count_vld +=1 

In [86]:
add_vld

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

In [87]:
add_vld_list[0]

[]

In [88]:
add_vld[16,23] # 16번째 고객의 23번째 상품이 신규구매이므로

1.0

In [89]:
add_vld_list[16] 

[23]

In [90]:
count_vld # 신규 구매 건수 

37897

In [0]:
# import mapk  # 왜 안될까
# average precision 
def apk(actual, predicted, k=7, default=0.0):
    # MAP@7, k=7
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits =0.0 

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]: # 정답이 있고 중복이 아니면
            num_hits += 1.0
            score += num_hits / (i+1.0)
    
    if not actual:
        return default

    return score / min(len(actual),k)   

def mapk(actual, predicted, k=7, default=0.0):
    return np.mean([apk(a,p,k,default) for a,p in zip(actual,predicted)]) # 고객별 apk 평균

In [94]:
# 검증데이터의 MAP@7 최고점 : 0.042663
print(mapk(add_vld_list,add_vld_list))

0.04266379915553903


In [0]:
# 검증데이터에 대한 예측값
X_vld = vld.as_matrix(columns=features)
Y_vld = vld.as_matrix(columns=['y'])
dvld = xgb.DMatrix(X_vld, label=Y_vld, feature_names=features)
preds_vld = model.predict(dvld, ntree_limit=best_ntree_limit)

In [95]:
preds_vld # 확률

array([[ 1.71149509e-06,  7.84041913e-06, -9.99954695e-01, ...,
         4.33766134e-02,  3.57372537e-02,  5.35317421e-01],
       [ 1.47602225e-06,  6.76170930e-06, -9.99960271e-01, ...,
         2.83613522e-02,  2.62700804e-02,  6.08545244e-01],
       [ 6.78548940e-06,  3.10845644e-05,  1.51783764e-01, ...,
        -9.98658303e-01, -9.98997754e-01, -9.99463987e-01],
       ...,
       [ 2.42349529e-06,  1.11021172e-05, -9.99925631e-01, ...,
         6.65091053e-02,  6.74702227e-02,  6.20828927e-01],
       [ 6.28272460e-07,  2.87813828e-06, -9.99976481e-01, ...,
         1.20594623e-02,  1.48870889e-02,  9.19652224e-01],
       [ 1.25857150e-06,  5.76555976e-06,  7.88953185e-01, ...,
         2.37722695e-02,  2.54630875e-02,  2.82873083e-02]])

In [0]:
# 신규 구매라고 예측하였으나 한달 전 (이미) 구매한 제품은 제외해야함
# 이 경우 prods_vld 확률에서 1 뺄셈
preds_vld = preds_vld - vld.as_matrix(columns=[prod+'_prev' for prod in prods])

In [96]:
# 참고
list(zip([1,2,3],['a','b','c']))

[(1, 'a'), (2, 'b'), (3, 'c')]

In [0]:
# 검증 데이터 예측 상위 7개 추출
result_vld = []
for ncodper, pred in zip(ncodpers_vld, preds_vld):
    y_prods = [(y,p,ip) for y,p,ip in zip(pred, prods,range(len(prods)))]
    y_prods = sorted(y_prods, key=lambda a: a[0], reverse=True)[:7]
    result_vld.append([ip for y,p,ip in y_prods])

print(mapk(add_vld_list, result_vld))

Baseline 모델 정확도 = 검증데이터 MAP@7 / 검증데이터 최고 점수 

: 어떻게 판단하는것일까? 

In [97]:
0.036466 / 0.042663 #85%

0.8547453296767691

In [98]:
# 위 코드 예시
ncodper = ncodpers_vld[0]
pred = preds_vld[0]
y_prods = list(zip(pred, prods, range(len(prods))))
y_prods # 확률, 상품명, 상품번호

[(1.7114950878749369e-06, 'ind_ahor_fin_ult1', 0),
 (7.840419129934162e-06, 'ind_aval_fin_ult1', 1),
 (-0.9999546948056377, 'ind_cco_fin_ult1', 2),
 (0.0003286047140136361, 'ind_cder_fin_ult1', 3),
 (0.0182532649487257, 'ind_cno_fin_ult1', 4),
 (1.0874562576645985e-05, 'ind_ctju_fin_ult1', 5),
 (0.0035789289977401495, 'ind_ctma_fin_ult1', 6),
 (0.013079753145575523, 'ind_ctop_fin_ult1', 7),
 (0.03224855288863182, 'ind_ctpp_fin_ult1', 8),
 (7.840419129934162e-06, 'ind_deco_fin_ult1', 9),
 (1.7114950878749369e-06, 'ind_deme_fin_ult1', 10),
 (0.014728052541613579, 'ind_dela_fin_ult1', 11),
 (0.12283958494663239, 'ind_ecue_fin_ult1', 12),
 (0.0014466814463958144, 'ind_fond_fin_ult1', 13),
 (3.246355845476501e-05, 'ind_hip_fin_ult1', 14),
 (0.001074545318260789, 'ind_plan_fin_ult1', 15),
 (2.762900840025395e-05, 'ind_pres_fin_ult1', 16),
 (0.008486397564411163, 'ind_reca_fin_ult1', 17),
 (0.16382117569446564, 'ind_tjcr_fin_ult1', 18),
 (0.005490692798048258, 'ind_valo_fin_ult1', 19),
 (5.71

In [99]:
sorted(y_prods, key=lambda a: a[0], reverse=True)[:7] # pred 기준으로 확률 내림차순, 상위 7개

[(0.5353174209594727, 'ind_recibo_ult1', 23),
 (0.16382117569446564, 'ind_tjcr_fin_ult1', 18),
 (0.12283958494663239, 'ind_ecue_fin_ult1', 12),
 (0.0433766134083271, 'ind_nomina_ult1', 21),
 (0.03573725372552872, 'ind_nom_pens_ult1', 22),
 (0.03224855288863182, 'ind_ctpp_fin_ult1', 8),
 (0.0182532649487257, 'ind_cno_fin_ult1', 4)]

In [100]:
result_vld[0:5] # 예측된 상품 번호

[[23, 18, 12, 21, 22, 8, 4],
 [23, 18, 12, 21, 22, 11, 4],
 [8, 17, 2, 19, 7, 11, 15],
 [23, 18, 12, 7, 21, 13, 22],
 [23, 12, 21, 4, 22, 13, 19]]

### 4. 전체 데이터 학습 및 테스트 데이터 예측

In [0]:
X_all = XY.as_matrix(columns=features)
Y_all = XY.as_matrix(columns=['y'])
dall = xgb.DMatrix(X_all, label=Y_all, feature_names=features)
watch_list = [(dall, 'train')]
# 트리 개수는 늘어난 데이터 양 만큼 비례해서 증가
best_ntree_limit = int(best_ntree_limit * (len(XY_trn)+len(XY_vld))/len(XY_trn))
# 재학습
model = xgb.train(param, dall, num_boost_round=best_ntree_limit, evals=watch_list) # 엄청 오래걸리네 ...

In [0]:
# 모델 저장
filename = 'xgb.baseline.all.pkl'
pickle.dump(model, open(filename,'wb'))

In [101]:
[(k,v) for k,v in model.get_fscore().items()]

[('ind_empleado_prev', 158),
 ('ind_cco_fin_ult1_prev', 3471),
 ('tiprel_1mes', 1903),
 ('ind_cno_fin_ult1_prev', 3461),
 ('ind_tjcr_fin_ult1_prev', 2566),
 ('ind_nomina_ult1_prev', 1679),
 ('ind_ctma_fin_ult1_prev', 1025),
 ('ind_ctju_fin_ult1_prev', 107),
 ('antiguedad', 25660),
 ('ind_ecue_fin_ult1_prev', 3798),
 ('age_prev', 18163),
 ('ind_ctop_fin_ult1_prev', 1892),
 ('age', 23348),
 ('renta', 27350),
 ('antiguedad_prev', 20663),
 ('ind_valo_fin_ult1_prev', 1789),
 ('canal_entrada_prev', 6665),
 ('segmento', 2609),
 ('ind_nuevo', 867),
 ('ind_actividad_cliente', 1321),
 ('segmento_prev', 2568),
 ('ind_actividad_cliente_prev', 991),
 ('ind_reca_fin_ult1_prev', 2526),
 ('ind_hip_fin_ult1_prev', 617),
 ('nomprov_prev', 9420),
 ('ind_ctpp_fin_ult1_prev', 1571),
 ('canal_entrada', 10067),
 ('ind_plan_fin_ult1_prev', 706),
 ('sexo', 4294),
 ('indext_prev', 512),
 ('nomprov', 14228),
 ('indrel_1mes_prev', 324),
 ('tiprel_1mes_prev', 1498),
 ('indrel_1mes', 453),
 ('ind_nuevo_prev', 574),

In [102]:
# 변수중요도 - EDA 결과와 비교해볼것
print("Feature importance:")
for kv in sorted([(k,v) for k,v in model.get_fscore().items()], key=lambda kv: kv[1], reverse=True):
    print(kv)

Feature importance:
('renta', 27350)
('antiguedad', 25660)
('age', 23348)
('antiguedad_prev', 20663)
('age_prev', 18163)
('nomprov', 14228)
('renta_prev', 12721)
('canal_entrada', 10067)
('nomprov_prev', 9420)
('canal_entrada_prev', 6665)
('sexo', 4294)
('ind_recibo_ult1_prev', 4000)
('ind_ecue_fin_ult1_prev', 3798)
('ind_cco_fin_ult1_prev', 3471)
('ind_cno_fin_ult1_prev', 3461)
('segmento', 2609)
('segmento_prev', 2568)
('ind_tjcr_fin_ult1_prev', 2566)
('ind_reca_fin_ult1_prev', 2526)
('tiprel_1mes', 1903)
('ind_nom_pens_ult1_prev', 1900)
('ind_ctop_fin_ult1_prev', 1892)
('ind_dela_fin_ult1_prev', 1814)
('ind_valo_fin_ult1_prev', 1789)
('sexo_prev', 1763)
('ind_nomina_ult1_prev', 1679)
('ind_ctpp_fin_ult1_prev', 1571)
('tiprel_1mes_prev', 1498)
('ind_actividad_cliente', 1321)
('ind_fond_fin_ult1_prev', 1117)
('ind_ctma_fin_ult1_prev', 1025)
('ind_actividad_cliente_prev', 991)
('indext', 950)
('ind_nuevo', 867)
('ind_plan_fin_ult1_prev', 706)
('ind_hip_fin_ult1_prev', 617)
('ind_nuevo_

In [0]:
# 테스트 데이터 예측 값
X_tst = tst.as_matrix(columns=features)
dtst = xgb.DMatrix(X_tst, feature_names=features)
preds_tst = model.predict(dtst, ntree_limit=best_ntree_limit)
ncodpers_tst = tst.as_matrix(columns=['ncodpers']) # 왜 필요하지?
preds_tst = preds_tst - tst.as_matrix(columns=[prod + '_prev' for prod in prods]) # 지난 달에 구입한 상품인 경우 방지

제출파일에 관한 경진대회 규칙

: 고객 식별 번호(ncodpers)와 7개의 제품 변수 이름을 공백 기준으로 띄워 입력할 것


In [0]:
# 제출 파일 생성
submit_file = open('../xgb.baseline.2015-06-28', 'w')
submit_file.write('ncodpers,added_products\n')
for ncodper, pred in zip(ncodpers_tst, preds_tst):
    y_prods = [(y,p,ip) for y,p,ip in zip(pred, prods, range(len(prods)))]
    y_prods = sorted(y_prods, key=lambda a: a[0], reverse=True)[:7] # 확률 제일 큰 7개
    y_prods = [p for y,p,ip in y_prods] # prods name만 추출
    submit_file.write('{},{}\n'.format(int(ncodper), ' '.join(y_prods))) # 무슨 코드인지 모르겠다 

In [0]:
y_prods = ['A','B','C']
'*'.join(y_prods)

'A*B*C'

In [0]:
# 위 코드 참고
list(zip(ncodpers_tst, preds_tst))[0]

(array([15889]),
 array([ 1.17063764e-05,  1.17063764e-05, -9.99902844e-01,  1.76199386e-03,
         8.06581508e-03,  1.96998863e-05,  4.55672474e-04,  1.08683445e-04,
        -9.99614105e-01,  1.17063764e-05,  1.44100989e-06,  7.44636776e-03,
         2.37477034e-01,  1.61055364e-02,  1.82707212e-04,  2.21204082e-03,
         4.04217775e-04,  2.14105118e-02, -9.99607326e-01, -9.99033336e-01,
         1.30406697e-04,  3.77478041e-02,  2.88833976e-02,  6.35709167e-01]))

In [0]:
# submit_file = open('xgb.baseline.2015-06-28', 'r')
lines = submit_file.readlines()
lines[0:5]

['15889,ind_recibo_ult1 ind_ecue_fin_ult1 ind_nomina_ult1 ind_nom_pens_ult1 ind_reca_fin_ult1 ind_fond_fin_ult1 ind_cno_fin_ult1\n',
 '1170544,ind_recibo_ult1 ind_cno_fin_ult1 ind_nom_pens_ult1 ind_nomina_ult1 ind_ecue_fin_ult1 ind_reca_fin_ult1 ind_tjcr_fin_ult1\n',
 '1170545,ind_recibo_ult1 ind_nom_pens_ult1 ind_nomina_ult1 ind_cno_fin_ult1 ind_tjcr_fin_ult1 ind_reca_fin_ult1 ind_ecue_fin_ult1\n',
 '1170547,ind_recibo_ult1 ind_reca_fin_ult1 ind_nom_pens_ult1 ind_cno_fin_ult1 ind_nomina_ult1 ind_ecue_fin_ult1 ind_ctma_fin_ult1\n',
 '1170548,ind_recibo_ult1 ind_reca_fin_ult1 ind_nom_pens_ult1 ind_cno_fin_ult1 ind_nomina_ult1 ind_ecue_fin_ult1 ind_dela_fin_ult1\n']